# Predicting diet proportions of all known extinct bird species using Random Forest

## PART 1. DATASET PRE-PROCESSING

This script contains the code used to pre-process the datasets to train a RF model.

### Loading packages

First, the necessary Python libraries are imported. These include pandas for data manipulation, scikit-learn for machine learning functionalities, matplotlib and seaborn for data visualization, and numpy for numerical operations.

In [301]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np

# 1. Pre-processing of AVOTREX dataset

## 1.1 Load AVOTREX dataset

We used a comprehensive dataset that combines information on the morphological traits for all known extinct species obtained from Sayol, F. et al., (2024). 

The code reads an excel file containing bird data using pd.read_xlsx() and stores it in a pandas DataFrame named data.

In [521]:
url = "https://github.com/martinezrubio/ECOTREX/raw/refs/heads/main/data/raw/AVOTREX_2025.csv"
data = pd.read_csv(url)
data.head()

,species,status,order,family,island_endemicity,region,archipelago,lat,long,flight_ability,...,beak_depth,tarsus_length,wing_length,kipps_distance,tail_length,body_mass,body_imputation_type,n_emm,n_skel,n_body
0,Accipiter_efficax,EP,Accipitriformes,Accipitridae,insular,Central_Indo_Pacific,New_Caledonia,-21.369993,166.149178,1.0,...,15.39,68.57,262.09,78.84,187.87,402.94,Imputed,0,5,0
1,Accipiter_quartus,EP,Accipitriformes,Accipitridae,insular,Central_Indo_Pacific,New_Caledonia,-21.369993,166.149178,1.0,...,12.12,53.76,209.15,62.99,149.68,211.21,Imputed,0,4,0
2,Accipiter_sp_Abaco,EP,Accipitriformes,Accipitridae,insular,Tropical_Atlantic,Bahamas_Turks_and_Caicos,23.358701,-75.150565,1.0,...,10.25,46.29,176.06,54.60,127.71,130.60,Estimated_from_descriptions,0,0,1
3,Acrocephalus_astrolabii,EX,Passeriformes,Acrocephalidae,insular,Eastern_Indo_Pacific,Gambier_Islands,-22.430041,-135.430670,1.0,...,4.81,33.00,100.00,16.56,83.00,33.65,Imputed,4,0,0
4,Acrocephalus_luscinius,EX,Passeriformes,Acrocephalidae,insular,Central_Indo_Pacific,Mariana_Islands,15.992530,145.811136,1.0,...,4.94,29.95,85.65,14.96,81.95,34.61,Imputed,8,0,0


In [491]:
data.shape # Should be 610 rows and 25 columns

(610, 25)

In [522]:
data.columns

Index(['species', 'status', 'order', 'family', 'island_endemicity', 'region',
       'archipelago', 'lat', 'long', 'flight_ability',
       'flight_imputation_type', 'skin_imputation_type', 'beak_length_culmen',
       'beak_length_nares', 'beak_width', 'beak_depth', 'tarsus_length',
       'wing_length', 'kipps_distance', 'tail_length', 'body_mass',
       'body_imputation_type', 'n_emm', 'n_skel', 'n_body'],
      dtype='object')

### 1.1.1 Remove non-informative variables

Non-informative metadata variables not used as predictors were removed to simplify the dataset prior to analysis.

In [523]:
cols_to_drop = [
    "status",
    "region",
    "archipelago",
    "lat",
    "long",
    "flight_imputation_type", 
    "skin_imputation_type",     
    "beak_length_nares", 
    "body_imputation_type",
    "n_emm",
    "n_skel",
    "n_body"
]

In [524]:
data = data.drop(columns=cols_to_drop)

In [525]:
data.columns

Index(['species', 'order', 'family', 'island_endemicity', 'flight_ability',
       'beak_length_culmen', 'beak_width', 'beak_depth', 'tarsus_length',
       'wing_length', 'kipps_distance', 'tail_length', 'body_mass'],
      dtype='object')

In [526]:
data.shape # Should be 610 rows and 13 columns

(610, 13)

### 1.1.2 Log-transformation of morphological traits

Morphological traits were log-transformed to account for allometric scaling and reduce skewness prior to analysis.

In [527]:
vars_to_log = [
    "beak_length_culmen",
    "beak_width",
    "beak_depth",
    "tarsus_length",
    "wing_length",
    "kipps_distance",
    "tail_length",
    "body_mass"
]

for var in vars_to_log:
    data[f"log_{var}"] = np.log(data[var])

In [528]:
data[[f"log_{v}" for v in vars_to_log]].describe()

,log_beak_length_culmen,log_beak_width,log_beak_depth,log_tarsus_length,log_wing_length,log_kipps_distance,log_tail_length,log_body_mass
count,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000
mean,3.570531,2.090740,2.383639,3.766418,5.052857,3.662368,4.569557,5.708373
std,0.595484,0.678614,0.682596,0.662124,1.092838,1.274608,0.649340,1.911856
min,1.684545,0.559616,0.470004,1.420696,-2.302585,-2.302585,2.332144,1.004302
25%,3.102342,1.556037,1.836573,3.302389,4.649331,3.086943,4.117002,4.350981
50%,3.557916,1.994699,2.329711,3.630455,5.114755,3.824933,4.534962,5.581122
75%,3.932945,2.647946,2.901971,4.165421,5.607766,4.501974,5.001510,6.725034
max,5.801695,3.995629,4.389250,6.180389,6.714049,5.684124,6.255596,13.360015


In [529]:
cols_to_drop = [
    "beak_length_culmen",
    "beak_width",
    "beak_depth",
    "tarsus_length",
    "wing_length",
    "kipps_distance",  
    "tail_length",      
    "body_mass"    
]

In [530]:
data = data.drop(columns=cols_to_drop)

In [531]:
data.columns

Index(['species', 'order', 'family', 'island_endemicity', 'flight_ability',
       'log_beak_length_culmen', 'log_beak_width', 'log_beak_depth',
       'log_tarsus_length', 'log_wing_length', 'log_kipps_distance',
       'log_tail_length', 'log_body_mass'],
      dtype='object')

In [532]:
data = data.rename(columns={
    "species": "Species",
    "order": "Order",
    "family": "Family",
    "island_endemicity" : "Island_Endemicity",
    "flight_ability" : "Flight_Ability",
    "log_beak_length_culmen" : "log_Beak.Length_Culmen",
    "log_beak_width" : "log_Beak.Width",
    "log_beak_depth" : "log_Beak.Depth",
    "log_tarsus_length" : "log_Tarsus.Length",
    "log_wing_length" : "log_Wing.Length",
    "log_kipps_distance" : "log_Kipps.Distance",
    "log_tail_length" : "log_Tail.Length",
    "log_body_mass" : "log_Mass"
})

In [533]:
data.columns

Index(['Species', 'Order', 'Family', 'Island_Endemicity', 'Flight_Ability',
       'log_Beak.Length_Culmen', 'log_Beak.Width', 'log_Beak.Depth',
       'log_Tarsus.Length', 'log_Wing.Length', 'log_Kipps.Distance',
       'log_Tail.Length', 'log_Mass'],
      dtype='object')

## 1.2 Incorporating sea birds classification and simple diet expected

In [447]:
url = "https://github.com/martinezrubio/ECOTREX/raw/refs/heads/main/data/raw/AVOTREX_seabird_dietsimple.xlsx"
diet = pd.read_excel(url)
diet.head()

,Species,seabird,Diet_Simple_Expected
0,Accipiter_efficax,no,Vertivore
1,Accipiter_quartus,no,Vertivore
2,Accipiter_sp_Abaco,no,Vertivore
3,Acrocephalus_astrolabii,no,Invertivore
4,Acrocephalus_luscinius,no,Invertivore


In [504]:
diet.shape # Should be 610 rows and 3 columns 

(610, 3)

In [534]:
data = data.merge(
    diet,
    on="Species",
    how="left"
)

In [535]:
data.columns

Index(['Species', 'Order', 'Family', 'Island_Endemicity', 'Flight_Ability',
       'log_Beak.Length_Culmen', 'log_Beak.Width', 'log_Beak.Depth',
       'log_Tarsus.Length', 'log_Wing.Length', 'log_Kipps.Distance',
       'log_Tail.Length', 'log_Mass', 'seabird', 'Diet_Simple_Expected'],
      dtype='object')

In [536]:
data.shape # Should be 610 rows and 15 columns 

(610, 15)

## 1.3 Incorporation of taxonomy bridge

For extinct species, taxonomic proximity information was incorporated by identifying the closest extant relatives at the family and order levels.

In [456]:
url = "https://raw.githubusercontent.com/martinezrubio/ECOTREX/main/data/raw/AVOTREX_Taxonomy_Bridge.xlsx"
taxonomy = pd.read_excel(url)
taxonomy.head()

,Species,Family,Family_bridge,Order,Order_bridge
0,Accipiter_efficax,Accipitridae,Accipitridae,Accipitriformes,Accipitriformes
1,Accipiter_quartus,Accipitridae,Accipitridae,Accipitriformes,Accipitriformes
2,Accipiter_sp_Abaco,Accipitridae,Accipitridae,Accipitriformes,Accipitriformes
3,Acrocephalus_astrolabii,Acrocephalidae,Acrocephalidae,Passeriformes,Passeriformes
4,Acrocephalus_luscinius,Acrocephalidae,Acrocephalidae,Passeriformes,Passeriformes


In [537]:
taxonomy.shape # Should be 610 rows and 5 columns

(610, 5)

In [538]:
taxonomy.columns

Index(['Species', 'Family', 'Family_bridge', 'Order', 'Order_bridge'], dtype='object')

**Selecting diet variables**

In [539]:
taxonomy_vars = [
    "Family_bridge","Order_bridge"
]

In [540]:
tax_sub = taxonomy[["Species"] + taxonomy_vars]

In [541]:
data = data.merge(
    tax_sub,
    on="Species",
    how="left"
)

In [542]:
data.columns

Index(['Species', 'Order', 'Family', 'Island_Endemicity', 'Flight_Ability',
       'log_Beak.Length_Culmen', 'log_Beak.Width', 'log_Beak.Depth',
       'log_Tarsus.Length', 'log_Wing.Length', 'log_Kipps.Distance',
       'log_Tail.Length', 'log_Mass', 'seabird', 'Diet_Simple_Expected',
       'Family_bridge', 'Order_bridge'],
      dtype='object')

In [543]:
data.shape # Should be 610 rows and 17 columns

(610, 17)

## 1.4 Phylogenetic PCA and integration of evolutionary information

Phylogenetic structure was accounted for by computing a Brownian-motion variance–covariance matrix from an avian phylogeny and extracting the first 12 phylogenetic principal components. These components summarize shared evolutionary history and were merged with the main dataset at the species level.

**Install R and rpy2**

In [72]:
!pip install biopython openpyxl
from Bio import Phylo
import openpyxl
from io import StringIO
import requests

In [73]:
url = "https://raw.githubusercontent.com/martinezrubio/ECOTREX/main/data/raw/ecotrex_tree.nwk"
tree_str = requests.get(url).text
tree = Phylo.read(StringIO(tree_str), "newick")

In [80]:
conda install -c conda-forge rpy2 r-base r-essentials

Retrieving notices: ...working... done
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
WARNING conda.models.version:get_matcher(556): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.8.0.*, but conda is ignoring the .* and treating it as 1.8.0
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.11.1



## Package Plan ##

  environment location: /Users/nataliamartinezrubio/anaconda3

  added / updated specs:
    - r-base
    - r-essentials
    - rpy2


The following packages will 

r-backports-1.5.0    | 126 KB    |                                       |   0% 
libclang-cpp14-14.0. | 13.1 MB   |                                       |   0% 

r-rcpp-1.0.10        | 1.9 MB    |                                       |   0% 


tzlocal-5.3.1        | 23 KB     |                                       |   0% 



r-cachem-1.1.0       | 73 KB     |                                       |   0% 




r-rematch-2.0.0      | 24 KB     |                                       |   0% 





r-survival-3.7_0     | 6.0 MB    |                                       |   0% 






r-openssl-2.1.1      | 670 KB    |                                       |   0% 







r-clipr-0.8.0        | 67 KB     |                                       |   0% 








r-numderiv-2016.8_1. | 125 KB    |                                       |   0% 









r-broom-1.0.6        | 1.7 MB    |                                       |   0% 










r-bit64-4.0.5        | 470 KB    |                    

r-prodlim-2023.03.31 | 421 KB    | #4                                    |   4% 


















r-prodlim-2023.03.31 | 421 KB    | ##################################### | 100% 

r-rcpp-1.0.10        | 1.9 MB    | ##################################### | 100% 

r-rcpp-1.0.10        | 1.9 MB    | ##################################### | 100% 




















r-nlme-3.1_162       | 2.2 MB    | 2                                     |   1% 





















r-sourcetools-0.1.7_ | 50 KB     | ###########7                          |  32% 





















r-sourcetools-0.1.7_ | 50 KB     | ##################################### | 100% 






















 ... (more hidden) ...




















r-nlme-3.1_162       | 2.2 MB    | ###########################4          |  74% 




















r-nlme-3.1_162       | 2.2 MB    | ##################################### | 100% 






















 ... (more hidden) ...
libclang-cpp14-14.0. | 13.1 MB   | ########################

Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


**Enable R execution inside the notebook (rpy2)**

In [35]:
%load_ext rpy2.ipython

**Compute phylogenetic PCA**

In [36]:
%%R -o pcs
library(ape)

tree <- read.tree("https://raw.githubusercontent.com/martinezrubio/ECOTREX/main/data/raw/ecotrex_tree.nwk")
V <- vcv(tree)
pca <- prcomp(V, scale.=TRUE)

pcs <- as.data.frame(pca$x[, 1:12])
colnames(pcs) <- paste0("PC", 1:12)
pcs$species <- rownames(V)

In [37]:
pcs.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,species
Megalapteryx_didinus,-206.274554,-108.923832,-1.743762,-0.317601,3.892841,-0.483482,0.660844,-3.653352,13.554580,-76.396884,85.771716,-10.984549,Megalapteryx_didinus
Euryapteryx_curtus,-206.291422,-108.951216,-1.745061,-0.317972,3.898758,-0.484266,0.662108,-3.662704,13.602923,-76.694778,86.145266,-11.048494,Euryapteryx_curtus
Emeus_crassus,-206.291422,-108.951216,-1.745061,-0.317972,3.898758,-0.484266,0.662108,-3.662704,13.602923,-76.694778,86.145266,-11.048494,Emeus_crassus
Anomalopteryx_didiformis,-206.290785,-108.950181,-1.745012,-0.317958,3.898534,-0.484236,0.662060,-3.662350,13.601096,-76.683523,86.131152,-11.046078,Anomalopteryx_didiformis
Pachyornis_geranoides,-206.289058,-108.947378,-1.744879,-0.317920,3.897929,-0.484156,0.661931,-3.661393,13.596148,-76.653032,86.092918,-11.039533,Pachyornis_geranoides


**Merge phylogenetic PCs with the main dataset**

In [544]:
data = data.merge(pcs, left_on="Species", right_on="species", how="left").drop(columns=["species"])

In [545]:
data.columns

Index(['Species', 'Order', 'Family', 'Island_Endemicity', 'Flight_Ability',
       'log_Beak.Length_Culmen', 'log_Beak.Width', 'log_Beak.Depth',
       'log_Tarsus.Length', 'log_Wing.Length', 'log_Kipps.Distance',
       'log_Tail.Length', 'log_Mass', 'seabird', 'Diet_Simple_Expected',
       'Family_bridge', 'Order_bridge', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5',
       'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12'],
      dtype='object')

## 1.5 Final dataset assembly

Relevant taxonomic, morphological, ecological, dietary, spatial, and phylogenetic variables were selected and explicitly ordered to define the final analysis-ready dataset used in downstream modelling.

In [546]:
final_columns = [
    "Species", "Family", "Order", "Family_bridge", "Order_bridge",
    "Island_Endemicity", "Flight_Ability",
    "log_Beak.Length_Culmen", "log_Beak.Width", "log_Beak.Depth",
    "log_Tarsus.Length", "log_Wing.Length", "log_Kipps.Distance", "log_Tail.Length", "log_Mass",
    "Diet_Simple_Expected","seabird",

    "PC1", "PC2", "PC3", "PC4", "PC5", "PC6",
    "PC7", "PC8", "PC9", "PC10", "PC11", "PC12"
]

In [547]:
data = data[final_columns]

In [548]:
data.columns.tolist()

['Species',
 'Family',
 'Order',
 'Family_bridge',
 'Order_bridge',
 'Island_Endemicity',
 'Flight_Ability',
 'log_Beak.Length_Culmen',
 'log_Beak.Width',
 'log_Beak.Depth',
 'log_Tarsus.Length',
 'log_Wing.Length',
 'log_Kipps.Distance',
 'log_Tail.Length',
 'log_Mass',
 'Diet_Simple_Expected',
 'seabird',
 'PC1',
 'PC2',
 'PC3',
 'PC4',
 'PC5',
 'PC6',
 'PC7',
 'PC8',
 'PC9',
 'PC10',
 'PC11',
 'PC12']

In [549]:
cols_to_add = [
    "IAE", "ISA", "ISS", "ISG", "IVS", "IGE", "IGG",
    "APG", "APP", "APA", "APL", "APS", "APD",
    "FAE", "FGL", "FGR",
    "NAE", "NGL", "SEL", "SGR",
    "PEL", "PGR", "PAG", "PAS", "PAD",
    "VAE", "VAS", "VPE", "VGE", "VGG",
    "CAQ", "CGR",
    "In", "Ap", "Vt", "Ne", "Fr", "Se", "Pa", "Pt",
    "dn.cat", "fd.cat"
]

In [550]:
for col in cols_to_add:
    if col not in data.columns:
        data[col] = np.nan

In [551]:
data.columns

Index(['Species', 'Family', 'Order', 'Family_bridge', 'Order_bridge',
       'Island_Endemicity', 'Flight_Ability', 'log_Beak.Length_Culmen',
       'log_Beak.Width', 'log_Beak.Depth', 'log_Tarsus.Length',
       'log_Wing.Length', 'log_Kipps.Distance', 'log_Tail.Length', 'log_Mass',
       'Diet_Simple_Expected', 'seabird', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5',
       'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'IAE', 'ISA', 'ISS',
       'ISG', 'IVS', 'IGE', 'IGG', 'APG', 'APP', 'APA', 'APL', 'APS', 'APD',
       'FAE', 'FGL', 'FGR', 'NAE', 'NGL', 'SEL', 'SGR', 'PEL', 'PGR', 'PAG',
       'PAS', 'PAD', 'VAE', 'VAS', 'VPE', 'VGE', 'VGG', 'CAQ', 'CGR', 'In',
       'Ap', 'Vt', 'Ne', 'Fr', 'Se', 'Pa', 'Pt', 'dn.cat', 'fd.cat'],
      dtype='object')

## 1.6 Save AVOTREX pre-processed dataset

In [552]:
data.to_csv("/Users/nataliamartinezrubio/Library/CloudStorage/GoogleDrive-natalia.maru3101@gmail.com/Mi unidad/Work/CREAF/A_Extinctions/F_EcoTrEx/C_RandomForest/F_GitHub/avotrex_processed.csv", index=False)